In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from datetime import datetime
from functools import reduce
import json
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto-gateway.serving.data.plectrum.dev'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [4]:
auto_full_zone = pd.read_csv('full_zone_auto.csv')
auto_full_zone_hexes = auto_full_zone['pickup_location_hex_8'].unique().tolist()
auto_inner_zone = pd.read_csv('inner_zone_auto.csv')
auto_inner_zone_hexes = auto_inner_zone['pickup_location_hex_8'].unique().tolist()
print(auto_inner_zone_hexes)

['8861892429fffff', '886189242dfffff', '8861892421fffff', '886189255bfffff', '8861892553fffff', '8861892467fffff', '8861892461fffff', '8861892469fffff', '8861892463fffff', '886189246bfffff', '8861892445fffff', '886189246dfffff', '8861892465fffff']


In [29]:
print(auto_full_zone_hexes)

['886189251bfffff', '8861892429fffff', '886189242dfffff', '8861892421fffff', '8861892427fffff', '886189240dfffff', '886189242bfffff', '8861892423fffff', '886189255bfffff', '8861892553fffff', '8861892467fffff', '8861892405fffff', '8861892461fffff', '88618925c9fffff', '8861892469fffff', '8861892463fffff', '8861892425fffff', '8861892519fffff', '8861892551fffff', '886189246bfffff', '8861892445fffff', '886189246dfffff', '8861892557fffff', '8861892465fffff']


In [6]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, city_name, customer_location_hex_8, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, eta, map_riders, pickup_location_hex_8, captain_obj_mobile, updated_epoch, captain_id
from orders.order_logs_snapshot
where yyyymmdd >= '20230417'
and yyyymmdd <= '20230423'
and city_name in ('Bangalore')
and service_obj_service_name in ('Auto')
and pickup_location_hex_8 in ('8861892429fffff', '886189242dfffff', '8861892421fffff', '886189255bfffff', '8861892553fffff', '8861892467fffff', '8861892461fffff',
 '8861892469fffff', '8861892463fffff', '886189246bfffff', '8861892445fffff', '886189246dfffff', '8861892465fffff')
and hour in ('17', '18', '19', '20', '21')
"""


#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [44]:
#Create a copy of the df retrived from presto
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_obj_mobile,updated_epoch,captain_id
0,6443fe390f2b53161ffec4e1,NaN,Bangalore,8861892553fffff,12.917195,77.648125,210313,21,2100,Auto,21:00,6,20230422,customerCancelled,order cancelled before rider accepted,0.0,"[""5b0392d34c87115d44d292e1"",""628f80176aed53422...",8861892553fffff,None,1682177613975,60ebceade5cb357d755a2675
1,644400406992a230ab9b8d80,0.615,Bangalore,8861892553fffff,12.917909,77.651851,211152,21,2100,Auto,21:00,6,20230422,dropped,None,3.0,"[""5c8bf6fc5e042733c9cb93fa"",""629b40622fdccd5fe...",8861892553fffff,9742517010,1682178631591,629b40622fdccd5fe9ab8390
2,6444037d5028ee299f9ba1e8,NaN,Bangalore,8861892421fffff,12.915257,77.635376,212541,21,2115,Auto,21:00,6,20230422,expired,None,0.0,"[""603ce8df4b0307455e98208f"",""60f7b6368f347953f...",8861892421fffff,None,1682179144046,611a44e634b93d770d2a544c
3,6444037d98dea70b775a5237,1.161,Bangalore,8861892553fffff,12.920215,77.648897,212541,21,2115,Auto,21:00,6,20230422,customerCancelled,Found better price elsewhere,5.0,"[""63689e81a00a915881aa0e17"",""5c8bf6fc5e042733c...",8861892553fffff,9980280252,1682179151478,61039b8a6cd4923d954026c7
4,644405b8df41c0543f168422,1.416,Bangalore,886189242dfffff,12.907383,77.645895,213512,21,2130,Auto,21:59,6,20230422,dropped,None,3.0,"[""629b40622fdccd5fe9ab8390"",""5c8bf6fc5e042733c...",886189242dfffff,9986302720,1682180234716,615406162421ab3bd60a3b58


In [45]:
#View the dataset
df_captain = df.copy()
df_captain.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_obj_mobile,updated_epoch,captain_id
0,6443fe390f2b53161ffec4e1,NaN,Bangalore,8861892553fffff,12.917195,77.648125,210313,21,2100,Auto,21:00,6,20230422,customerCancelled,order cancelled before rider accepted,0.0,"[""5b0392d34c87115d44d292e1"",""628f80176aed53422...",8861892553fffff,None,1682177613975,60ebceade5cb357d755a2675
1,644400406992a230ab9b8d80,0.615,Bangalore,8861892553fffff,12.917909,77.651851,211152,21,2100,Auto,21:00,6,20230422,dropped,None,3.0,"[""5c8bf6fc5e042733c9cb93fa"",""629b40622fdccd5fe...",8861892553fffff,9742517010,1682178631591,629b40622fdccd5fe9ab8390
2,6444037d5028ee299f9ba1e8,NaN,Bangalore,8861892421fffff,12.915257,77.635376,212541,21,2115,Auto,21:00,6,20230422,expired,None,0.0,"[""603ce8df4b0307455e98208f"",""60f7b6368f347953f...",8861892421fffff,None,1682179144046,611a44e634b93d770d2a544c
3,6444037d98dea70b775a5237,1.161,Bangalore,8861892553fffff,12.920215,77.648897,212541,21,2115,Auto,21:00,6,20230422,customerCancelled,Found better price elsewhere,5.0,"[""63689e81a00a915881aa0e17"",""5c8bf6fc5e042733c...",8861892553fffff,9980280252,1682179151478,61039b8a6cd4923d954026c7
4,644405b8df41c0543f168422,1.416,Bangalore,886189242dfffff,12.907383,77.645895,213512,21,2130,Auto,21:59,6,20230422,dropped,None,3.0,"[""629b40622fdccd5fe9ab8390"",""5c8bf6fc5e042733c...",886189242dfffff,9986302720,1682180234716,615406162421ab3bd60a3b58


In [46]:
#Filter only on the hexes needed
df_filter_level1 = df
df_filter_level1['count'] = 1
df_filter_level1['date'] = df_filter_level1['yyyymmdd'].astype(str).apply(lambda x: x[6:])

In [47]:
#Create a new column for cancel_reason to identify OCARA
df_filter_level1['new_cancel_reason'] = np.where(df_filter_level1['order_status'] == 'customerCancelled',
 np.where(df_filter_level1['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
 np.where(df_filter_level1['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

In [48]:
#Check if the order is expiry_mapped or expiry_unmapped
df_filter_level1['expiry_type'] = np.where(df_filter_level1['order_status'] == 'expired',
 np.where(df_filter_level1['map_riders'] == '[]', 'expiry_unmapped', 'expiry_mapped'), 'no_expiry')
df_filter_level1['final_order_status'] = np.where(df_filter_level1['order_status'] == 'customerCancelled', df_filter_level1['new_cancel_reason'],
np.where(df_filter_level1['order_status'] == 'expired', df_filter_level1['expiry_type'], df_filter_level1['order_status']))

In [49]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd >= '20230417'
and yyyymmdd <= '20230423'
and service_obj_service_name = 'Auto'
and event_type in ('accepted', 'rider_busy', 'rider_reject')
and city_name = 'Bangalore'
and pickup_location_hex_8 in ('8861892429fffff', '886189242dfffff', '8861892421fffff', '886189255bfffff', '8861892553fffff', '8861892467fffff', '8861892461fffff',
 '8861892469fffff', '8861892463fffff', '886189246bfffff', '8861892445fffff', '886189246dfffff', '8861892465fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [50]:
df_pings = df_apr_copy.copy()
df_pings = df_pings.sort_values(by = 'updated_epoch', ascending = False)
df_pings = df_pings.drop_duplicates(subset = 'order_id')
df_filter_level1 = df_filter_level1.merge(df_pings[['order_id', 'captain_location_hex_8']], on = 'order_id', how = 'left')
df_filter_level1.head()

,order_id,accept_to_pickup_distance,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,eta,map_riders,pickup_location_hex_8,captain_obj_mobile,updated_epoch,captain_id,count,date,new_cancel_reason,expiry_type,final_order_status,captain_location_hex_8
0,6443fe390f2b53161ffec4e1,NaN,Bangalore,8861892553fffff,12.917195,77.648125,210313,21,2100,Auto,21:00,6,20230422,customerCancelled,order cancelled before rider accepted,0.0,"[""5b0392d34c87115d44d292e1"",""628f80176aed53422...",8861892553fffff,None,1682177613975,60ebceade5cb357d755a2675,1,22,COBRA,no_expiry,COBRA,8861892553fffff
1,644400406992a230ab9b8d80,0.615,Bangalore,8861892553fffff,12.917909,77.651851,211152,21,2100,Auto,21:00,6,20230422,dropped,None,3.0,"[""5c8bf6fc5e042733c9cb93fa"",""629b40622fdccd5fe...",8861892553fffff,9742517010,1682178631591,629b40622fdccd5fe9ab8390,1,22,no_cancellation,no_expiry,dropped,8861892553fffff
2,6444037d5028ee299f9ba1e8,NaN,Bangalore,8861892421fffff,12.915257,77.635376,212541,21,2115,Auto,21:00,6,20230422,expired,None,0.0,"[""603ce8df4b0307455e98208f"",""60f7b6368f347953f...",8861892421fffff,None,1682179144046,611a44e634b93d770d2a544c,1,22,no_cancellation,expiry_mapped,expiry_mapped,8861892423fffff
3,6444037d98dea70b775a5237,1.161,Bangalore,8861892553fffff,12.920215,77.648897,212541,21,2115,Auto,21:00,6,20230422,customerCancelled,Found better price elsewhere,5.0,"[""63689e81a00a915881aa0e17"",""5c8bf6fc5e042733c...",8861892553fffff,9980280252,1682179151478,61039b8a6cd4923d954026c7,1,22,OCARA,no_expiry,OCARA,886189255bfffff
4,644405b8df41c0543f168422,1.416,Bangalore,886189242dfffff,12.907383,77.645895,213512,21,2130,Auto,21:59,6,20230422,dropped,None,3.0,"[""629b40622fdccd5fe9ab8390"",""5c8bf6fc5e042733c...",886189242dfffff,9986302720,1682180234716,615406162421ab3bd60a3b58,1,22,no_cancellation,no_expiry,dropped,8861892429fffff


In [51]:
df_filter_level1['captain_location_hex_8'] = df_filter_level1['captain_location_hex_8'].fillna('no_cap_location')

In [52]:
#Metrics_pivot
metrics_pivot = pd.pivot_table(df_filter_level1, index = ['pickup_location_hex_8', 'captain_location_hex_8'], columns = 'final_order_status',
 values = 'count', aggfunc = 'sum', fill_value=0).reset_index().rename_axis(None, axis=1)
metrics_pivot['total'] = metrics_pivot[['COBRA', 'OCARA', 'aborted', 'dropped', 'expiry_mapped']].sum(axis = 1)
metrics_pivot['OCARA%'] = round(metrics_pivot['OCARA']/metrics_pivot['total'], 4)
metrics_pivot['COBRA%'] = round(metrics_pivot['COBRA']/metrics_pivot['total'], 4)
metrics_pivot['COBRM%'] = 0
metrics_pivot['G2N'] = round(metrics_pivot['dropped']/metrics_pivot['total'], 4)
metrics_pivot['expiry_mapped%'] = round(metrics_pivot['expiry_mapped']/metrics_pivot['total'], 4)
if 'expiry_unmapped' in metrics_pivot.columns:
    metrics_pivot['Stockout%'] = round(metrics_pivot['expiry_unmapped']/metrics_pivot['total'], 4)
else:
    metrics_pivot['Stockout%'] = 0
metrics_df = metrics_pivot[['pickup_location_hex_8', 'captain_location_hex_8', 'OCARA%', 'COBRA%', 'COBRM%', 'G2N', 'expiry_mapped%',
 'Stockout%', 'total']]
metrics_df.head()

,pickup_location_hex_8,captain_location_hex_8,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total
0,8861892421fffff,,0.2179,0.0000,0,0.7782,0.0000,0,257
1,8861892421fffff,8861892405fffff,0.0451,0.2481,0,0.0526,0.6541,0,133
2,8861892421fffff,8861892407fffff,0.0000,0.2000,0,0.0000,0.8000,0,10
3,8861892421fffff,8861892421fffff,0.0720,0.2658,0,0.3971,0.2645,0,3002
4,8861892421fffff,8861892423fffff,0.0541,0.2735,0,0.1735,0.4949,0,980


In [53]:
#Get the average ETA and FM
df_dropped = df_filter_level1[df_filter_level1['final_order_status'] == 'dropped']
fm = df_dropped.groupby(['pickup_location_hex_8', 'captain_location_hex_8'])['accept_to_pickup_distance'].median().reset_index()
fm = fm.rename(columns = {'accept_to_pickup_distance':'FM'})
eta = df_dropped.groupby(['pickup_location_hex_8', 'captain_location_hex_8'])['eta'].median().reset_index()
metrics_df = metrics_df.merge(fm, on = ['pickup_location_hex_8', 'captain_location_hex_8'], how = 'left')
metrics_df = metrics_df.merge(eta, on = ['pickup_location_hex_8', 'captain_location_hex_8'], how = 'left')
metrics_df.head()

,pickup_location_hex_8,captain_location_hex_8,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta
0,8861892421fffff,,0.2179,0.0000,0,0.7782,0.0000,0,257,0.4580,2.0
1,8861892421fffff,8861892405fffff,0.0451,0.2481,0,0.0526,0.6541,0,133,1.0010,5.0
2,8861892421fffff,8861892407fffff,0.0000,0.2000,0,0.0000,0.8000,0,10,NaN,NaN
3,8861892421fffff,8861892421fffff,0.0720,0.2658,0,0.3971,0.2645,0,3002,0.5010,2.0
4,8861892421fffff,8861892423fffff,0.0541,0.2735,0,0.1735,0.4949,0,980,0.9195,4.0


In [54]:
metrics_df['FM'] = metrics_df['FM'].fillna('no_dropped_order')
metrics_df['eta'] = metrics_df['eta'].fillna('no_dropped_order')

In [55]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd >= '20230417'
and yyyymmdd <= '20230423'
and service_obj_service_name = 'Auto'
and event_type in ('accepted', 'rider_busy', 'rider_reject', 'dropped')
and city_name = 'Bangalore'
and pickup_location_hex_8 in ('8861892429fffff', '886189242dfffff', '8861892421fffff', '886189255bfffff', '8861892553fffff', '8861892467fffff', '8861892461fffff',
 '8861892469fffff', '8861892463fffff', '886189246bfffff', '8861892445fffff', '886189246dfffff', '8861892465fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [56]:
#Load and view dataset
df_apr = df_apr_copy.copy()
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8
0,64451d335028ee299f9d5646,NaN,886189255bfffff,12.911951,77.650520,Bangalore,886189242dfffff,8861892437fffff,12.913249,77.608956,12.912213,77.642479,172739,17,1715,Auto,17:00,7,20230423,new,None,1682251121748,rider_busy,63da55eefbbdfd41f3b85053,None,886189242dfffff
1,64451e4b44636e16f0838056,NaN,8861892467fffff,12.903618,77.649318,Bangalore,886189255bfffff,8861892e9bfffff,12.974782,77.606602,12.908959,77.650665,173219,17,1730,Auto,17:59,7,20230423,new,None,1682251376392,rider_reject,6108dd7bdf98a523ff890e27,None,886189255bfffff
2,64451eb75028ee299f9d599e,NaN,8861892467fffff,12.907698,77.649183,Bangalore,8861892429fffff,886014596dfffff,13.050769,77.570910,12.906240,77.641258,173407,17,1730,Auto,17:59,7,20230423,new,None,1682251488511,rider_reject,62a6eea317af9ac363562667,None,8861892429fffff
3,64451f3be56df743451f6941,NaN,8861892427fffff,12.915556,77.632099,Bangalore,8861892421fffff,8861892437fffff,12.909724,77.614273,12.913543,77.638252,173619,17,1730,Auto,17:59,7,20230423,new,None,1682251583940,rider_reject,6120a75401c2c371593d1912,None,8861892421fffff
4,64451f9d5028ee299f9d5b90,NaN,8861892553fffff,12.917146,77.644686,Bangalore,886189242dfffff,88618925c1fffff,12.925581,77.615067,12.909062,77.640999,173757,17,1730,Auto,17:59,7,20230423,new,None,1682251717969,rider_reject,6328343ecca85b3387ac4ede,None,886189242dfffff


In [57]:
#Create a count column
df_apr['count'] = 1

#Create a column to identify total pings and accepted ping
df_apr['total_pings'] = np.where(df_apr['event_type'].isin(['accepted', 'rider_busy', 'rider_reject']), 1, 0)
df_apr['accepted_pings'] = np.where(df_apr['event_type'].isin(['accepted']), 1, 0)
df_apr['rider_busy_reject'] = np.where(df_apr['event_type'].isin(['rider_busy', 'rider_reject']), 1, 0)
df_me = df_apr[(df_apr['order_status'] == 'dropped') & (df_apr['spd_fraud_flag'] != True)]

#Groupby on date and hour to get the APR and ME on an hourly basis
apr_hyderabad = df_apr.groupby(['pickup_location_hex_8', 'captain_location_hex_8'])[['total_pings', 'accepted_pings',
 'rider_busy_reject']].sum().reset_index()
apr_hyderabad['APR'] = round(apr_hyderabad['accepted_pings']/apr_hyderabad['total_pings'], 4)
apr_hyderabad['rejected_pings'] = round(apr_hyderabad['rider_busy_reject']/apr_hyderabad['total_pings'], 4)
me_hyderabad = df_me.groupby(['pickup_location_hex_8', 'captain_location_hex_8'])['order_id'].nunique().reset_index()
me_hyderabad = me_hyderabad.rename(columns={'order_id':'net_orders'})
apr_hyderabad = apr_hyderabad.merge(me_hyderabad, on = ['pickup_location_hex_8', 'captain_location_hex_8'], how = 'left')
apr_hyderabad['ME'] = round(apr_hyderabad['total_pings']/apr_hyderabad['net_orders'], 2)

#Merge APR and ME with metrics_df
metrics_df = metrics_df.merge(apr_hyderabad, on = ['pickup_location_hex_8', 'captain_location_hex_8'], how = 'left')
metrics_df.head()

,pickup_location_hex_8,captain_location_hex_8,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,total_pings,accepted_pings,rider_busy_reject,APR,rejected_pings,net_orders,ME
0,8861892421fffff,,0.2179,0.0000,0,0.7782,0.0000,0,257,0.458,2.0,274.0,274.0,0.0,1.0000,0.0000,NaN,NaN
1,8861892421fffff,8861892405fffff,0.0451,0.2481,0,0.0526,0.6541,0,133,1.001,5.0,788.0,15.0,773.0,0.0190,0.9810,44.0,17.91
2,8861892421fffff,8861892407fffff,0.0000,0.2000,0,0.0000,0.8000,0,10,no_dropped_order,no_dropped_order,53.0,0.0,53.0,0.0000,1.0000,3.0,17.67
3,8861892421fffff,8861892421fffff,0.0720,0.2658,0,0.3971,0.2645,0,3002,0.501,2.0,31069.0,1556.0,29513.0,0.0501,0.9499,140.0,221.92
4,8861892421fffff,8861892423fffff,0.0541,0.2735,0,0.1735,0.4949,0,980,0.9195,4.0,8685.0,256.0,8429.0,0.0295,0.9705,133.0,65.30


In [58]:
apr_hyderabad['sourcing_side'] = np.where(apr_hyderabad['captain_location_hex_8'].isin(auto_full_zone_hexes), 'in_sourced', 'out_sourced')
apr_hyderabad.head()

,pickup_location_hex_8,captain_location_hex_8,total_pings,accepted_pings,rider_busy_reject,APR,rejected_pings,net_orders,ME,sourcing_side
0,8861892421fffff,,274,274,0,1.0,0.0,NaN,NaN,out_sourced
1,8861892421fffff,886014584bfffff,0,0,0,NaN,NaN,1.0,0.0,out_sourced
2,8861892421fffff,8860145a07fffff,0,0,0,NaN,NaN,1.0,0.0,out_sourced
3,8861892421fffff,8860145a09fffff,0,0,0,NaN,NaN,1.0,0.0,out_sourced
4,8861892421fffff,8860145a0dfffff,0,0,0,NaN,NaN,2.0,0.0,out_sourced


In [59]:
metrics_df['sourcing_side'] = np.where(metrics_df['captain_location_hex_8'].isin(auto_full_zone_hexes), 'in_sourced', 'out_sourced')
metrics_df.head()

,pickup_location_hex_8,captain_location_hex_8,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,total_pings,accepted_pings,rider_busy_reject,APR,rejected_pings,net_orders,ME,sourcing_side
0,8861892421fffff,,0.2179,0.0000,0,0.7782,0.0000,0,257,0.458,2.0,274.0,274.0,0.0,1.0000,0.0000,NaN,NaN,out_sourced
1,8861892421fffff,8861892405fffff,0.0451,0.2481,0,0.0526,0.6541,0,133,1.001,5.0,788.0,15.0,773.0,0.0190,0.9810,44.0,17.91,in_sourced
2,8861892421fffff,8861892407fffff,0.0000,0.2000,0,0.0000,0.8000,0,10,no_dropped_order,no_dropped_order,53.0,0.0,53.0,0.0000,1.0000,3.0,17.67,out_sourced
3,8861892421fffff,8861892421fffff,0.0720,0.2658,0,0.3971,0.2645,0,3002,0.501,2.0,31069.0,1556.0,29513.0,0.0501,0.9499,140.0,221.92,in_sourced
4,8861892421fffff,8861892423fffff,0.0541,0.2735,0,0.1735,0.4949,0,980,0.9195,4.0,8685.0,256.0,8429.0,0.0295,0.9705,133.0,65.30,in_sourced


In [60]:
apr = apr_hyderabad.groupby(['sourcing_side'])[['total_pings', 'accepted_pings', 'rider_busy_reject']].sum().reset_index()
apr['APR'] = apr['accepted_pings']/apr['total_pings']
apr['rider_busy_reject%'] = apr['rider_busy_reject']/apr['total_pings']
apr

,sourcing_side,total_pings,accepted_pings,rider_busy_reject,APR,rider_busy_reject%
0,in_sourced,480553,17190,463363,0.035771,0.964229
1,out_sourced,9676,1060,8616,0.109549,0.890451


In [61]:
metrics_df['OCARA_orders'] = metrics_df['total']*metrics_df['OCARA%']

In [62]:
ocara = metrics_df[(metrics_df['total'] >= 100)].groupby(['sourcing_side'])[['OCARA_orders', 'total']].sum().reset_index()
ocara['ocara%'] = ocara['OCARA_orders']/ocara['total']
ocara

,sourcing_side,OCARA_orders,total,ocara%
0,in_sourced,2831.7767,52769,0.053664
1,out_sourced,139.0028,2074,0.067022


In [64]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type, captain_id, spd_fraud_flag, pickup_location_hex_8
from orders.order_logs_immutable
where yyyymmdd >= '20230417'
and yyyymmdd <= '20230418'
and service_obj_service_name = 'Auto'
and event_type in ('accepted', 'rider_busy', 'rider_reject', 'dropped')
and city_name = 'Bangalore'
and pickup_location_hex_8 not in ('886189251bfffff', '8861892429fffff', '886189242dfffff', '8861892421fffff', '8861892427fffff', '886189240dfffff', '886189242bfffff',
 '8861892423fffff', '886189255bfffff', '8861892553fffff', '8861892467fffff', '8861892405fffff', '8861892461fffff', '88618925c9fffff', '8861892469fffff',
   '8861892463fffff', '8861892425fffff', '8861892519fffff', '8861892551fffff', '886189246bfffff', '8861892445fffff', '886189246dfffff', '8861892557fffff',
     '8861892465fffff')
and hour in ('17', '18', '19', '20', '21')
"""

#Load data into pandas table
df_apr_copy = pd.read_sql(q, connection)

In [65]:
#Load and view dataset
df_apr = df_apr_copy.copy()
df_apr.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,captain_id,spd_fraud_flag,pickup_location_hex_8
0,643ec229d90c1d22f8a3ab90,NaN,88618925ebfffff,12.937265,77.618386,Bangalore,8861892589fffff,88618925c7fffff,12.932924,77.614040,12.935696,77.608300,214537,21,2145,Auto,21:59,2,20230418,new,None,1681834612036,rider_busy,62411daf98d4a0b076cee24d,None,8861892589fffff
1,643ec501ef4e9f501b2b9304,NaN,88618925edfffff,12.948203,77.627813,Bangalore,88618925ebfffff,88618925ebfffff,12.941172,77.621305,12.940439,77.617943,215745,21,2145,Auto,21:59,2,20230418,onTheWay,None,1681835269740,accepted,5d30321b84e7394e94717d06,None,88618925e1fffff
2,643ec508792087485c415c28,NaN,8861892e91fffff,12.982058,77.610438,Bangalore,8861892e93fffff,8861892ed5fffff,12.976093,77.633751,12.983698,77.603706,215752,21,2145,Auto,21:59,2,20230418,new,None,1681835320511,rider_reject,61091ff61d7c300f91f93bc5,None,8861892e93fffff
3,643ec50c8d79690b458a8189,NaN,88618925ebfffff,12.942336,77.619708,Bangalore,88618925c7fffff,886189258dfffff,12.940665,77.612282,12.935480,77.612152,215756,21,2145,Auto,21:59,2,20230418,new,None,1681835316846,rider_busy,638ff3b81fc3fa68b318ddb3,None,88618925c7fffff
4,643ec532d90c1d22f8a3afdf,NaN,8861892c8bfffff,13.036077,77.595966,Bangalore,8861892c89fffff,8861892cc7fffff,13.039635,77.620628,13.033947,77.609123,215834,21,2145,Auto,21:59,2,20230418,new,None,1681835341938,rider_busy,60f143a0d417d413367931c4,None,8861892c89fffff


In [66]:
#Create a count column
df_apr['count'] = 1

#Create a column to identify total pings and accepted ping
df_apr['total_pings'] = np.where(df_apr['event_type'].isin(['accepted', 'rider_busy', 'rider_reject']), 1, 0)
df_apr['accepted_pings'] = np.where(df_apr['event_type'].isin(['accepted']), 1, 0)
df_apr['rider_busy_reject'] = np.where(df_apr['event_type'].isin(['rider_busy', 'rider_reject']), 1, 0)
df_me = df_apr[(df_apr['order_status'] == 'dropped') & (df_apr['spd_fraud_flag'] != True)]

#Groupby on date and hour to get the APR and ME on an hourly basis
apr_hyderabad = df_apr.groupby(['pickup_location_hex_8', 'captain_location_hex_8'])[['total_pings', 'accepted_pings',
 'rider_busy_reject']].sum().reset_index()
apr_hyderabad['APR'] = round(apr_hyderabad['accepted_pings']/apr_hyderabad['total_pings'], 4)
apr_hyderabad['rejected_pings'] = round(apr_hyderabad['rider_busy_reject']/apr_hyderabad['total_pings'], 4)
me_hyderabad = df_me.groupby(['pickup_location_hex_8', 'captain_location_hex_8'])['order_id'].nunique().reset_index()
me_hyderabad = me_hyderabad.rename(columns={'order_id':'net_orders'})
apr_hyderabad = apr_hyderabad.merge(me_hyderabad, on = ['pickup_location_hex_8', 'captain_location_hex_8'], how = 'left')
apr_hyderabad['ME'] = round(apr_hyderabad['total_pings']/apr_hyderabad['net_orders'], 2)

#Merge APR and ME with metrics_df
metrics_df = metrics_df.merge(apr_hyderabad, on = ['pickup_location_hex_8', 'captain_location_hex_8'], how = 'left')
metrics_df.head()

,pickup_location_hex_8,captain_location_hex_8,OCARA%,COBRA%,COBRM%,G2N,expiry_mapped%,Stockout%,total,FM,eta,total_pings_x,accepted_pings_x,rider_busy_reject_x,APR_x,rejected_pings_x,net_orders_x,ME_x,sourcing_side,OCARA_orders,total_pings_y,accepted_pings_y,rider_busy_reject_y,APR_y,rejected_pings_y,net_orders_y,ME_y
0,8861892421fffff,,0.2179,0.0000,0,0.7782,0.0000,0,257,0.458,2.0,274.0,274.0,0.0,1.0000,0.0000,NaN,NaN,out_sourced,56.0003,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8861892421fffff,8861892405fffff,0.0451,0.2481,0,0.0526,0.6541,0,133,1.001,5.0,788.0,15.0,773.0,0.0190,0.9810,44.0,17.91,in_sourced,5.9983,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8861892421fffff,8861892407fffff,0.0000,0.2000,0,0.0000,0.8000,0,10,no_dropped_order,no_dropped_order,53.0,0.0,53.0,0.0000,1.0000,3.0,17.67,out_sourced,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8861892421fffff,8861892421fffff,0.0720,0.2658,0,0.3971,0.2645,0,3002,0.501,2.0,31069.0,1556.0,29513.0,0.0501,0.9499,140.0,221.92,in_sourced,216.1440,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8861892421fffff,8861892423fffff,0.0541,0.2735,0,0.1735,0.4949,0,980,0.9195,4.0,8685.0,256.0,8429.0,0.0295,0.9705,133.0,65.30,in_sourced,53.0180,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
apr_hyderabad['sourcing_side'] = np.where(apr_hyderabad['captain_location_hex_8'].isin(auto_inner_zone_hexes), 'inside_zone_hexes', 'outside_zone_hexes')
apr_hyderabad.head()

,pickup_location_hex_8,captain_location_hex_8,total_pings,accepted_pings,rider_busy_reject,APR,rejected_pings,net_orders,ME,sourcing_side
0,8860144b41fffff,8860144b4dfffff,5,0,5,0.0000,1.0000,NaN,NaN,outside_zone_hexes
1,8860144b49fffff,8860144b49fffff,1,0,1,0.0000,1.0000,NaN,NaN,outside_zone_hexes
2,8860144b49fffff,88601459a3fffff,1,1,0,1.0000,0.0000,NaN,NaN,outside_zone_hexes
3,8860144b4dfffff,8860144b41fffff,2,0,2,0.0000,1.0000,NaN,NaN,outside_zone_hexes
4,8860144b4dfffff,8860144b4dfffff,73,12,61,0.1644,0.8356,NaN,NaN,outside_zone_hexes


In [68]:
apr = apr_hyderabad.groupby(['sourcing_side'])[['total_pings', 'accepted_pings', 'rider_busy_reject']].sum().reset_index()
apr['APR'] = apr['accepted_pings']/apr['total_pings']
apr['rider_busy_reject%'] = apr['rider_busy_reject']/apr['total_pings']
apr

,sourcing_side,total_pings,accepted_pings,rider_busy_reject,APR,rider_busy_reject%
0,inside_zone_hexes,1717,96,1621,0.055911,0.944089
1,outside_zone_hexes,1907899,61039,1846860,0.031993,0.968007


In [1]:
1717/(1717+1907899)

0.0008991336478119161